In [1]:
!pip install stable_baselines3[extra]
!pip install sb3-contrib

zsh:1: no matches found: stable_baselines3[extra]
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.8/171.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 MB 17.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 24.9 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616799 sha256=0cad8d8e8e02200db4a5a631dc5e226046bcb661d926251c5ea9fb63e4255ea9
  Stored in directory: /Users/tanchongmin/Library/Caches/pip/wheels/b3/50/6c/0a82c1358b4da2dbd9c1bb17e0f89467db32812ab236dbf6d5
Successfully built gym
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.11.3
    Uninstalling importlib-metadata-4.11.3:
      Successfully uninstalled importlib-metadata-4.11.3


In [4]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from queue import PriorityQueue
import threading
from threading import Thread

import stable_baselines3 as sb3
from stable_baselines3 import PPO, A2C, DQN, HerReplayBuffer
from sb3_contrib import ARS, TRPO, QRDQN
import torch

import gym
import numpy as np
from gym import spaces

print(tf.__version__)

# Define some sample next states and actions to try to learn by supervised learning

In [ ]:
def getnextaction(state):
  cur_x, cur_y, goal_x, goal_y = state
  if cur_x > goal_x:
    return 0
  elif cur_x < goal_x:
    return 1
  elif cur_y > goal_y:
    return 2
  elif cur_y < goal_y:
    return 3
  else:
    return 4

In [ ]:
# this is used for the alternate formulation
def getnextaction2(state):
  cur_x, cur_y, goal_x, goal_y = state
  if cur_y > goal_y:
    return 0
  elif cur_y < goal_y:
    return 1
  elif cur_x > goal_x:
    return 2
  elif cur_x < goal_x:
    return 3
  else:
    return 4

In [ ]:
def getnextstate(state):
  cur_x, cur_y, goal_x, goal_y = state
  if cur_x > goal_x:
    return (cur_x-1, cur_y)
  elif cur_x < goal_x:
    return (cur_x+1, cur_y)
  elif cur_y > goal_y:
    return (cur_x, cur_y - 1)
  elif cur_y < goal_y:
    return (cur_x, cur_y+1)
  else:
    return (cur_x, cur_y)

In [ ]:
# this is used for the alternate formulation
def getnextstate2(state):
  cur_x, cur_y, goal_x, goal_y = state
  if cur_y > goal_y:
    return (cur_x, cur_y - 1)
  elif cur_y < goal_y:
    return (cur_x, cur_y+1)
  elif cur_x > goal_x:
    return (cur_x-1, cur_y)
  elif cur_x < goal_x:
    return (cur_x+1, cur_y)
  else:
    return (cur_x, cur_y)

In [ ]:
def GetSamples(grid_size = 10, seed = 0):
  np.random.seed(seed)
  states = np.random.randint(0, grid_size, (1000, 4))
  actions = np.array([getnextaction(state) for state in states])
  next_states = np.array([getnextstate(state) for state in states])

  actions2 = np.array([getnextaction2(state) for state in states])
  next_states2 = np.array([getnextstate2(state) for state in states])

  return states, actions, next_states, actions2, next_states2

# Experiment 1:
# Input: Current State, Goal State
# Output: Action

This shows that learning an action given the current state and conditioned on the goal state is easy.

In [ ]:
grid_size = 10
inputs = tf.keras.layers.Input(shape = (4,))
# x = tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3), strides=(1,1), padding='same', activation = tf.nn.relu)(inputs)
# x = tf.keras.layers.MaxPooling2D(pool_size = (2,2))(x)
# x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128, activation = tf.nn.relu)(inputs)
x = tf.keras.layers.Dense(128, activation = tf.nn.relu)(x)
output1 = tf.keras.layers.Dense(5, activation = tf.nn.softmax)(x)
 
model = tf.keras.Model(inputs = inputs, outputs = output1)

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

states, actions, next_states, actions2, next_states2 = GetSamples(grid_size = 10)

datasize = len(states)
split = datasize//5 * 4
# reshape the input to add a channel dimension at the end for CNN processing
history = model.fit(states[:split], actions[:split], epochs=50)

## swap x and y priority for the next state
datasize = len(states)
split = datasize//5 * 4
# reshape the input to add a channel dimension at the end for CNN processing
history2 = model.fit(states[:split], actions2[:split], epochs=50)

action10 = history.history['accuracy']+history2.history['accuracy']

In [ ]:
grid_size = 20
inputs = tf.keras.layers.Input(shape = (4,))
# x = tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3), strides=(1,1), padding='same', activation = tf.nn.relu)(inputs)
# x = tf.keras.layers.MaxPooling2D(pool_size = (2,2))(x)
# x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128, activation = tf.nn.relu)(inputs)
x = tf.keras.layers.Dense(128, activation = tf.nn.relu)(x)
output1 = tf.keras.layers.Dense(5, activation = tf.nn.softmax)(x)
 
model = tf.keras.Model(inputs = inputs, outputs = output1)

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

states, actions, next_states, actions2, next_states2 = GetSamples(grid_size = 20)

datasize = len(states)
split = datasize//5 * 4
history = model.fit(states[:split], actions[:split], epochs=50)

## swap x and y priority for the next state
datasize = len(states)
split = datasize//5 * 4
history2 = model.fit(states[:split], actions2[:split], epochs=50)

action20 = history.history['accuracy']+history2.history['accuracy']

In [ ]:
plt.plot(action10, 'b', label = 'Next Action Prediction 10x10')
plt.plot(action20, 'r', label = 'Next Action Prediction 20x20')
plt.xlabel('Number of Epochs')
plt.ylabel('Accuracy')
plt.xticks([i*10 for i in range(11)])
plt.grid()
plt.legend(loc = 'lower right')

# Experiment 2
# Input: Current State, Goal State
# Output: Next State

This shows that learning the next state conditioned on current state and goal state is difficult.

In [ ]:
grid_size = 10
inputs = tf.keras.layers.Input(shape = (4,))
# x = tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3), strides=(1,1), padding='same', activation = tf.nn.relu)(inputs)
# x = tf.keras.layers.MaxPooling2D(pool_size = (2,2))(x)
# x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128, activation = tf.nn.relu)(inputs)
x = tf.keras.layers.Dense(128, activation = tf.nn.relu)(x)
output1 = tf.keras.layers.Dense(grid_size, activation = tf.nn.softmax, name = 'state_x')(x)
output2 = tf.keras.layers.Dense(grid_size, activation = tf.nn.softmax, name = 'state_y')(x)
 
model2 = tf.keras.Model(inputs = inputs, outputs = [output1, output2])

model2.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

states, actions, next_states, actions2, next_states2 = GetSamples(grid_size = 10)

datasize = len(states)
split = datasize//5 * 4
history = model2.fit(states[:split], [next_states[:split][:,0], next_states[:split][:,1]], epochs=200)

## swap x and y priority for next state
datasize = len(states)
split = datasize//5 * 4
history2 = model2.fit(states[:split], [next_states2[:split][:,0], next_states2[:split][:,1]], epochs=200)

acc_x = history.history['state_x_accuracy']+history2.history['state_x_accuracy']
acc_y = history.history['state_y_accuracy']+history2.history['state_y_accuracy']
nextstate10 = (np.array(acc_x)+np.array(acc_y))/2

In [ ]:
grid_size = 20
inputs = tf.keras.layers.Input(shape = (4,))
# x = tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3), strides=(1,1), padding='same', activation = tf.nn.relu)(inputs)
# x = tf.keras.layers.MaxPooling2D(pool_size = (2,2))(x)
# x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128, activation = tf.nn.relu)(inputs)
x = tf.keras.layers.Dense(128, activation = tf.nn.relu)(x)
output1 = tf.keras.layers.Dense(grid_size, activation = tf.nn.softmax, name = 'state_x')(x)
output2 = tf.keras.layers.Dense(grid_size, activation = tf.nn.softmax, name = 'state_y')(x)
 
model2 = tf.keras.Model(inputs = inputs, outputs = [output1, output2])

model2.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

states, actions, next_states, actions2, next_states2 = GetSamples(grid_size = 20)

datasize = len(states)
split = datasize//5 * 4
history = model2.fit(states[:split], [next_states[:split][:,0], next_states[:split][:,1]], epochs=200)

## swap x and y priority for next state
datasize = len(states)
split = datasize//5 * 4
history2 = model2.fit(states[:split], [next_states2[:split][:,0], next_states2[:split][:,1]], epochs=200)

acc_x = history.history['state_x_accuracy']+history2.history['state_x_accuracy']
acc_y = history.history['state_y_accuracy']+history2.history['state_y_accuracy']
nextstate20 = (np.array(acc_x)+np.array(acc_y))/2

In [ ]:
plt.plot(nextstate10, 'b', label = 'Next State Prediction 10x10')
plt.plot(nextstate20, 'r', label = 'Next State Prediction 20x20')
plt.xticks([i*50 for i in range(9)])
plt.grid()
plt.xlabel('Number of Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Create Grid Environment

In [5]:
num_actions = 4

class GridEnv(gym.Env):
    """Custom Environment that follows gym interface."""

    metadata = {"render.modes": ["human"]}

    def __init__(self, size=5, start_state = None, goal_state = None, forbidden_squares = [], seed = None):
        super().__init__()

        # initialize action and observation space
        self.action_space = spaces.Discrete(4)
        self.observation_space = spaces.Box(low=0, high=size-1, shape=(4,), dtype=np.uint8)

        self.env_size = size
        self.forbidden_squares = forbidden_squares

        self.steps = 0

        if seed is not None:
          np.random.seed(seed)

        self.start_state = (np.random.randint(0, self.env_size), np.random.randint(0, self.env_size))
        self.goal_state = (np.random.randint(0, self.env_size), np.random.randint(0, self.env_size))

        # make sure we don't spawn in the forbidden squares
        while self.start_state in self.forbidden_squares:
          self.start_state = (np.random.randint(0, self.env_size), np.random.randint(0, self.env_size))
        # make sure we don't spawn in forbidden squares, and that the goal state is not the start state
        while self.goal_state in self.forbidden_squares or self.goal_state == self.start_state:
          self.goal_state = (np.random.randint(0, self.env_size), np.random.randint(0, self.env_size))
        if start_state is not None:
          self.start_state = start_state
        if goal_state is not None:
          self.goal_state = goal_state
        self.cur_state = self.start_state
        self.min_steps = self.get_min()

    def get_min(self):
        ''' Gets the minimum number of moves from start state to goal state using breadth first search '''
        visited = [self.start_state]
        cur_nodes = [self.start_state]
        count = 0
        while self.goal_state not in cur_nodes and count <= self.env_size * self.env_size:
          future_nodes = []
          count += 1 # increment 1 to the visited counts
          while len(cur_nodes) > 0:
            node = cur_nodes.pop()
            for future_node in [(node[0]-1, node[1]), (node[0]+1, node[1]), (node[0], node[1]-1), (node[0], node[1]+1)]:
              if future_node not in visited and 0 <= future_node[0] < self.env_size and 0 <= future_node[1] < self.env_size:
                visited.append(future_node)
                future_nodes.append(future_node)
          cur_nodes = future_nodes.copy()
        return count

    def step(self, action):
        # only step if episode is not terminated
        ''' Takes an action and also returns 1 if the goal state is reached '''
        initial_state = self.cur_state
        # left
        if action == 0 and self.cur_state[0] > 0:
          self.cur_state = (self.cur_state[0]-1, self.cur_state[1])
        # up
        elif action == 1 and self.cur_state[1] > 0:
          self.cur_state = (self.cur_state[0], self.cur_state[1]-1)
        # right
        elif action == 2 and self.cur_state[0]+1 < self.env_size:
          self.cur_state = (self.cur_state[0]+1, self.cur_state[1])
        # down
        elif action == 3 and self.cur_state[1]+1 < self.env_size:
          self.cur_state = (self.cur_state[0], self.cur_state[1]+1)
        # otherwise don't move
        # check if we hit the forbidden states. If so, reposition back to the beginning
        if self.cur_state in self.forbidden_squares:
          self.cur_state = initial_state
      
        # increment number of steps by 1
        observation = self.cur_state + self.goal_state
        self.steps += 1
        done = self.steps >= self.env_size * self.env_size # only stop once we reached end of the training steps

        reward = 0
        # terminate if we have reached the goal
        if self.cur_state == self.goal_state:
          reward = 1
          done = True

        return observation, reward, done, {}

    def reset(self):
        self.cur_state = self.start_state # reset start state
        observation = self.cur_state + self.goal_state
        self.steps = 0 # reset step count
        return observation  # reward, done, info can't be included

    def render(self, mode="human"):
        print(self.cur_state+self.goal_state)

    def close(self):
        ''' Don't have to close anything as we are not rendering anything '''
        return

## Helper Functions

In [6]:
def minion():
    ''' This references existing memory and tries to find a trajectory towards the goal state '''
    # this stores the list of states the minion experiences
    minion_state_history = [cur_state]
    minion_action_history = []
    for depth in range(lookahead_depth):
      # look for matches
      matches = []

      for action in range(num_actions):
        matches.extend([(state, action) for state in memory[minion_state_history[-1]][action]])

      # no match, then discontinue this minion's search
      if len(matches) == 0:
        break

      next_minion_state, next_minion_action = matches[np.random.randint(len(matches))]
      # update minion's value
      minion_state_history.append(next_minion_state)
      if depth==0:
        best_action = next_minion_action
      minion_action_history.append(next_minion_action)
      
      # check if goal state reached
      if minion_state_history[-1] == goal_state:
        sem.acquire()
        best_minions.put((depth, best_action, minion_state_history[:-1], minion_action_history, next_minion_action))
        sem.release()
        break

def GetModel(seed = 0):
    ''' This takes in a random seed and builds the desired TensorFlow model '''

    # set random seeds
    tf.random.set_seed(seed)
    np.random.seed(seed)

    inputs = tf.keras.layers.Input(shape = (4,))
    # x = tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3), strides=(1,1), padding='same', activation = tf.nn.relu)(inputs)
    # x = tf.keras.layers.MaxPooling2D(pool_size = (2,2))(x)
    # x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(128, activation = tf.nn.relu)(inputs)
    x = tf.keras.layers.Dense(128, activation = tf.nn.relu)(x)
    output1 = tf.keras.layers.Dense(num_actions, activation = tf.nn.softmax)(x)
    
    model = tf.keras.Model(inputs = inputs, outputs = output1)

    model.compile(optimizer = 'adam',
                  loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    
    return model

def GetGrid(episode, grid_size = 10):
  ''' This takes in an episode number and returns the desired forbidden squares'''
  # Define what is traversible
  if episode < 50:
    forbidden_squares = [(x, grid_size//2) for x in range(grid_size) if x!=grid_size//2]
  else:
    forbidden_squares = [(grid_size//2, x) for x in range(grid_size) if x!=grid_size//2]

  return forbidden_squares

# Experiment 1: Solving the grid with fixed start and end points

- We define a 10x10 grid and the agent is supposed to go from (0, 0) to (grid_size-1, grid_size - 1)
- No obstacles

## Model 1: PPO, TRPO, A2C


In [ ]:
for ALGO in [PPO, TRPO, A2C]:
    print (ALGO.__name__)
    grid_size = 10
    # Custom MLP policy of two layers of size 128 each with Relu activation function
    policy_kwargs = dict(activation_fn=torch.nn.ReLU, net_arch=[128, 128])
    # Create the environment
    env = GridEnv(size = grid_size, start_state = (0, 0), goal_state = (grid_size-1, grid_size-1), forbidden_squares = [])
    # Create the agent
    # set random seed
    sb3.common.utils.set_random_seed(0)
    if ALGO == A2C:
      model = ALGO("MlpPolicy", env, policy_kwargs=policy_kwargs, learning_rate = 0.0001)
    else:
      model = ALGO("MlpPolicy", env, policy_kwargs=policy_kwargs)
    model.save("my_env")

    best_steps = []
    actual_steps = []
    for i in range(100):
      # learn for each episode
      env = GridEnv(size = grid_size, start_state = (0, 0), goal_state = (grid_size-1, grid_size-1), forbidden_squares = [])
      model = ALGO.load("my_env", env=env)
      model.learn(total_timesteps=100)
      model.save("my_env")
      obs = env.reset()
      for steps in range(100):
        action, _states = model.predict(obs)
        obs, rewards, done, info = env.step(action)
        if done:
          if rewards == 1 and steps+1 < grid_size*grid_size:
            print(f'Run {i+1} completed in {steps+1} steps. Minimum steps: {env.min_steps}, Start state: {env.start_state}, Goal state: {env.goal_state}')

          else:
            print(f'Run {i+1} not completed. Minimum steps: {env.min_steps}, Start state: {env.start_state}, Goal state: {env.goal_state}')
          best_steps.append(env.min_steps)
          actual_steps.append(steps+1)
          break

    # save the best steps
    if ALGO == PPO:
      ppo_steps = actual_steps
    elif ALGO == TRPO:
      trpo_steps = actual_steps
    else:
      a2c_steps = actual_steps

## Model 2: Q-Learning Approach to solving the grid with fixed start and end points
$Q(s,a) = Q(s,a) + \alpha(r + \gamma\max_a Q(s',a) - Q(s,a))$

The maximum next state value only applies if it is not a terminal state.

Epsilon-greedy action selection, with greedy action being $argmax_a Q(s,a)$

In [ ]:
grid_size = 10
alpha = 1
gamma = 0.99
eps = 1
num_actions = 4
num_episodes = 100

best_steps = []
actual_steps = []
q_values = defaultdict(lambda: 0)
np.random.seed(0)
for i in range(num_episodes):
  env = GridEnv(grid_size, start_state = (0, 0), goal_state = (grid_size-1, grid_size-1), forbidden_squares = [])
  for steps in range(grid_size * grid_size):
    cur_state = env.cur_state
    action = 0
    # take random action if less than desired episode count
    if i<75:
      action = np.random.randint(num_actions)
    # otherwise choose the maximum action
    else:
      action_values = [q_values[(cur_state, a)] for a in range(num_actions)]
      action = np.argmax(action_values)

    obs, rewards, done, _ = env.step(action)
    next_state = (obs[0], obs[1])

    # update the Q value tables
    future_action_values = [q_values[(next_state, a)] for a in range(num_actions)]
    old_state_value = q_values[(cur_state, action)]
    if rewards == 1:
      q_values[(cur_state, action)] = old_state_value + alpha*(rewards - old_state_value)
    else:
      q_values[(cur_state, action)] = old_state_value + alpha*(rewards + gamma*np.max(future_action_values) - old_state_value)

    if done:
      if rewards == 1 and steps+1 < grid_size*grid_size:
        print(f'Run {i+1} completed in {steps+1} steps. Minimum steps: {env.min_steps}, Start state: {env.start_state}, Goal state: {env.goal_state}')

      else:
        print(f'Run {i+1} not completed. Minimum steps: {env.min_steps}, Start state: {env.start_state}, Goal state: {env.goal_state}')
      best_steps.append(env.min_steps)
      actual_steps.append(steps+1)
      break

qlearning_steps = actual_steps

## Model 3: Memory-based approach 

In [ ]:
# hyperparameters
use_fast = True
use_slow = True

grid_size = 10
exploration_const = 1
parallel_threads = 100
lookahead_depth = 20

# create model
model = GetModel()

# reset memory
memory = defaultdict(lambda: defaultdict(lambda: []))

# store only first action
best_steps = []
actual_steps = []

for i in range(100):
  env = GridEnv(grid_size, start_state = (0, 0), goal_state = (grid_size-1, grid_size-1), forbidden_squares = [])
  statehistory = []
  actionhistory = []
  episode_memory = defaultdict(lambda: defaultdict(lambda: []))

  for steps in range(grid_size*grid_size):
    cur_state = env.cur_state
    goal_state = env.goal_state

    # initialize training array
    start_states = []
    start_action = []
    # STEP 1: Fast neural network to get general cue for direction
    if use_fast:
      action_prob = model.predict(np.array(cur_state+goal_state).reshape(-1,4), verbose = 0)
      action_probs = action_prob[0]
    else:
      # set to 0s if not using the fast neural network
      action_probs = np.array([0 for _ in range(num_actions)], dtype = 'float')

    # STEP 2: Do explore-exploit to choose action

    # calculate the exploration parameter, take action_probs and add in count values from instances in memory
    action_probs -= [exploration_const*np.sqrt(len(episode_memory[env.cur_state][i])) for i in range(num_actions)]
    
    # choose the best action via explore-exploit tradeoff
    action = np.argmax(action_probs)

    # STEP 3: Forward Replay: find out if there is a path in memory to the goal (akin to Model-Based Planning)
    if use_slow:
      best_minions = PriorityQueue()
      best_action = 0

      threads = []
      sem = threading.Semaphore()
      for _ in range(parallel_threads):
        t = Thread(target=minion())
        threads.append(t)
        t.start()

      for t in threads:
        t.join()
      
      # Do memory action if found in memory. Also, do a replay to learn the transition of the start state to the end state
      if not best_minions.empty():
        depth, action, minion_state_history, minion_action_history, final_action = best_minions.get()
        # print(f'Found best action {action}. Depth {depth}. Current state {env.cur_state}.')

        if depth > 0:
          # update fast neural network with each state on future state trajectory as start state, and the final goal state as the goal state, with each action on future action trajectory
          start_states.extend([x+goal_state for x in minion_state_history])
          start_action.extend([int(x) for x in minion_action_history])

    # perform action
    obs, rewards, done, _= env.step(action)
    next_state = (obs[0], obs[1])

    # STEP 4: store transition in memory, delete irrelevant states if environment has changed
    for mem in [memory, episode_memory]:
      if len(mem[cur_state][action]) > 0 and mem[cur_state][action][-1] != next_state:
        mem[cur_state][action] = [next_state]
      else:
        mem[cur_state][action].append(next_state)

    # append current state and action to history
    statehistory.append(cur_state)
    actionhistory.append(action)

    # terminate if completed
    if done:
      if rewards == 1 and steps+1 < grid_size*grid_size:
        print(f'Run {i+1} completed in {steps+1} steps. Minimum steps: {env.min_steps}, Start state: {env.start_state}, Goal state: {env.goal_state}')

      else:
        print(f'Run {i+1} not completed. Minimum steps: {env.min_steps}, Start state: {env.start_state}, Goal state: {env.goal_state}')
      best_steps.append(env.min_steps)
      actual_steps.append(steps+1)

      break

    # STEP 5: Backward Replay: update fast neural network with next state as goal state, and states before as start state, with current action
    if use_fast:
      start_states.extend([x+next_state for x in statehistory])
      start_action.extend([int(x) for x in actionhistory])
      model.fit(start_states, start_action, epochs=1, verbose = False)

memory_steps = actual_steps

In [ ]:
# Across all states
plt.plot(memory_steps, 'b-', label = 'Fast & Slow')
plt.plot(ppo_steps, 'r-', label = 'PPO')
plt.plot(trpo_steps, 'y-', label = 'TRPO')
plt.plot(a2c_steps, 'g-', label = 'A2C')
plt.plot(qlearning_steps, 'c-', label = 'Q-Learning')
# plt.plot(best_steps, 'm--', label = 'Best')
plt.xlabel('Episode Number')
plt.ylabel('Number of Steps')
plt.legend(loc = 'upper right')
plt.show()

In [ ]:
# Only first 10 states
plt.plot(memory_steps[:10], 'b-', label = 'Fast & Slow')
plt.plot(ppo_steps[:10], 'r-', label = 'PPO')
plt.plot(trpo_steps[:10], 'y-', label = 'TRPO')
plt.plot(a2c_steps[:10], 'g-', label = 'A2C')
plt.plot(qlearning_steps[:10], 'c-', label = 'Q-Learning')
# plt.plot(best_steps[:10], 'm--', label = 'Best')
plt.xlabel('Episode Number')
plt.ylabel('Number of Steps')
plt.legend(loc = 'upper right')
plt.show()

In [ ]:
# This shows us the solve rate and steps above minimum for all methods
for steps in [memory_steps, ppo_steps, trpo_steps, a2c_steps, qlearning_steps]:
  actual_steps = np.array(steps)
  print(f'Number of episodes solved overall is {sum(actual_steps<grid_size*grid_size)}')
  print(f'Number of steps above minimum overall is {sum(actual_steps)-sum(best_steps)}')

# Experiment 2: Solving the grid with random start and end points
- Start and end goal differs each episode
- Obstacles will change every 50 episodes
- Agent has no pre-built model of the environment and has to lookahead without any external help
- Agent can rely on stored memory from start of the episodes, and memory can also carry over between episodes

## Model 1: PPO, TRPO, A2C

In [ ]:
for ALGO in [PPO, TRPO, A2C]:
    print (ALGO.__name__)
    grid_size = 40
    # Custom MLP policy of two layers of size 128 each with Relu activation function
    policy_kwargs = dict(activation_fn=torch.nn.ReLU, net_arch=[128, 128])
    # Create the environment
    i = 0
    env = GridEnv(grid_size, start_state = None, goal_state = None, forbidden_squares = GetGrid(i, grid_size), seed = i)
    # Create the agent
    # set random seed
    sb3.common.utils.set_random_seed(0)
    if ALGO == A2C:
      model = ALGO("MlpPolicy", env, policy_kwargs=policy_kwargs, learning_rate = 0.0001)
    else:
      model = ALGO("MlpPolicy", env, policy_kwargs=policy_kwargs)
    model.save("my_env")

    best_steps = []
    actual_steps = []
    for i in range(100):
      # learn for each episode
      env = GridEnv(grid_size, start_state = None, goal_state = None, forbidden_squares = GetGrid(i, grid_size), seed = i)
      model = ALGO.load("my_env", env=env)
      model.learn(total_timesteps=100)
      model.save("my_env")
      obs = env.reset()
      for steps in range(grid_size*grid_size):
        action, _states = model.predict(obs)
        obs, rewards, done, info = env.step(action)
        if done:
          if rewards == 1 and steps+1 < grid_size*grid_size:
            print(f'Run {i+1} completed in {steps+1} steps. Minimum steps: {env.min_steps}, Start state: {env.start_state}, Goal state: {env.goal_state}')

          else:
            print(f'Run {i+1} not completed. Minimum steps: {env.min_steps}, Start state: {env.start_state}, Goal state: {env.goal_state}')
          best_steps.append(env.min_steps)
          actual_steps.append(steps+1)

          break

    plt.plot(best_steps, 'b-', label = 'Minimum Steps per Episode')
    plt.plot(actual_steps, 'g-', label = 'Actual Steps per Episode')
    plt.xlabel('Episode number')
    plt.ylabel('Number of Steps')
    plt.xticks([i*10 for i in range(11)])
    # plt.legend(loc = 'upper right')
    plt.show()

    actual_steps = np.array(actual_steps)
    print(f'Number of steps above minimum for first 50 epochs is {sum(actual_steps[:50])-sum(best_steps[:50])}')
    print(f'Number of steps above minimum for last 50 epochs is {sum(actual_steps[50:])-sum(best_steps[50:])}')
    print(f'Number of steps above minimum overall is {sum(actual_steps)-sum(best_steps)}')
    print(f'Number of episodes solved for first 50 epochs is {sum(actual_steps[:50]<grid_size*grid_size)}')
    print(f'Number of episodes solved for last 50 epochs is {sum(actual_steps[50:]<grid_size*grid_size)}')
    print(f'Number of episodes solved overall is {sum(actual_steps<grid_size*grid_size)}')

    # save the best steps
    if ALGO == PPO:
      ppo_best_steps = best_steps
    elif ALGO == TRPO:
      trpo_steps = best_steps
    else:
      a2c_steps = best_steps

## Model 2: Fast and Slow

In [ ]:
# hyperparameters
use_fast = True
use_slow = True

grid_size = 10
exploration_const = 1
parallel_threads = 100
lookahead_depth = 20

# create model
model = GetModel()

# reset memory
memory = defaultdict(lambda: defaultdict(lambda: []))

# store only first action
best_steps = []
actual_steps = []

for i in range(100):
  env = GridEnv(grid_size, start_state = None, goal_state = None, forbidden_squares = GetGrid(i, grid_size), seed = i)
  statehistory = []
  actionhistory = []
  episode_memory = defaultdict(lambda: defaultdict(lambda: []))

  for steps in range(grid_size*grid_size):
    cur_state = env.cur_state
    goal_state = env.goal_state

    # initialize training array
    start_states = []
    start_action = []
    # STEP 1: Fast neural network to get general cue for direction
    if use_fast:
      action_prob = model.predict(np.array(cur_state+goal_state).reshape(-1,4), verbose = 0)
      action_probs = action_prob[0]
    else:
      # set to 0s if not using the fast neural network
      action_probs = np.array([0 for _ in range(num_actions)], dtype = 'float')

    # STEP 2: Do explore-exploit to choose action

    # calculate the exploration parameter, take action_probs and add in count values from instances in memory
    action_probs -= [exploration_const*np.sqrt(len(episode_memory[env.cur_state][i])) for i in range(num_actions)]
    
    # choose the best action via explore-exploit tradeoff
    action = np.argmax(action_probs)

    # STEP 3: Forward Replay: find out if there is a path in memory to the goal (akin to Model-Based Planning)
    if use_slow:
      best_minions = PriorityQueue()
      best_action = 0

      threads = []
      sem = threading.Semaphore()
      for _ in range(parallel_threads):
        t = Thread(target=minion())
        threads.append(t)
        t.start()

      for t in threads:
        t.join()
      
      # Do memory action if found in memory. Also, do a replay to learn the transition of the start state to the end state
      if not best_minions.empty():
        depth, action, minion_state_history, minion_action_history, final_action = best_minions.get()
        # print(f'Found best action {action}. Depth {depth}. Current state {env.cur_state}.')

        if depth > 0:
          # update fast neural network with each state on future state trajectory as start state, and the final goal state as the goal state, with each action on future action trajectory
          start_states.extend([x+goal_state for x in minion_state_history])
          start_action.extend([int(x) for x in minion_action_history])

    # perform action
    obs, rewards, done, _= env.step(action)
    next_state = (obs[0], obs[1])

    # STEP 4: store transition in memory, delete irrelevant states if environment has changed
    for mem in [memory, episode_memory]:
      if len(mem[cur_state][action]) > 0 and mem[cur_state][action][-1] != next_state:
        mem[cur_state][action] = [next_state]
      else:
        mem[cur_state][action].append(next_state)

    # append current state and action to history
    statehistory.append(cur_state)
    actionhistory.append(action)

    # terminate if completed
    if done:
      if rewards == 1 and steps+1 < grid_size*grid_size:
        print(f'Run {i+1} completed in {steps+1} steps. Minimum steps: {env.min_steps}, Start state: {env.start_state}, Goal state: {env.goal_state}')

      else:
        print(f'Run {i+1} not completed. Minimum steps: {env.min_steps}, Start state: {env.start_state}, Goal state: {env.goal_state}')
      best_steps.append(env.min_steps)
      actual_steps.append(steps+1)

      break

    # STEP 5: Backward Replay: update fast neural network with next state as goal state, and states before as start state, with current action
    if use_fast:
      start_states.extend([x+next_state for x in statehistory])
      start_action.extend([int(x) for x in actionhistory])
      model.fit(start_states, start_action, epochs=1, verbose = False)

In [ ]:
plt.plot(best_steps, 'b-', label = 'Minimum Steps per Episode')
plt.plot(actual_steps, 'g-', label = 'Actual Steps per Episode')
plt.xlabel('Episode number')
plt.ylabel('Number of Steps')
plt.xticks([i*10 for i in range(11)])
# plt.legend(loc = 'upper right')
plt.show()

actual_steps = np.array(actual_steps)
print(f'Number of steps above minimum for first 50 epochs is {sum(actual_steps[:50])-sum(best_steps[:50])}')
print(f'Number of steps above minimum for last 50 epochs is {sum(actual_steps[50:])-sum(best_steps[50:])}')
print(f'Number of steps above minimum overall is {sum(actual_steps)-sum(best_steps)}')
print(f'Number of episodes solved for first 50 epochs is {sum(actual_steps[:50]<grid_size*grid_size)}')
print(f'Number of episodes solved for last 50 epochs is {sum(actual_steps[50:]<grid_size*grid_size)}')
print(f'Number of episodes solved overall is {sum(actual_steps<grid_size*grid_size)}')

## Model 3: Without slow memory mechanism

In [ ]:
# hyperparameters
use_fast = True
use_slow = False

grid_size = 10
exploration_const = 1
parallel_threads = 100
lookahead_depth = 20

# create model
model = GetModel()

# reset memory
memory = defaultdict(lambda: defaultdict(lambda: []))

# store only first action
best_steps = []
actual_steps = []

for i in range(100):
  env = GridEnv(grid_size, start_state = None, goal_state = None, forbidden_squares = GetGrid(i, grid_size), seed = i)
  statehistory = []
  actionhistory = []
  episode_memory = defaultdict(lambda: defaultdict(lambda: []))

  for steps in range(grid_size*grid_size):
    cur_state = env.cur_state
    goal_state = env.goal_state

    # initialize training array
    start_states = []
    start_action = []
    # STEP 1: Fast neural network to get general cue for direction
    if use_fast:
      action_prob = model.predict(np.array(cur_state+goal_state).reshape(-1,4), verbose = 0)
      action_probs = action_prob[0]
    else:
      # set to 0s if not using the fast neural network
      action_probs = np.array([0 for _ in range(num_actions)], dtype = 'float')

    # STEP 2: Do explore-exploit to choose action

    # calculate the exploration parameter, take action_probs and add in count values from instances in memory
    action_probs -= [exploration_const*np.sqrt(len(episode_memory[env.cur_state][i])) for i in range(num_actions)]
    
    # choose the best action via explore-exploit tradeoff
    action = np.argmax(action_probs)

    # STEP 3: Forward Replay: find out if there is a path in memory to the goal (akin to Model-Based Planning)
    if use_slow:
      best_minions = PriorityQueue()
      best_action = 0

      threads = []
      sem = threading.Semaphore()
      for _ in range(parallel_threads):
        t = Thread(target=minion())
        threads.append(t)
        t.start()

      for t in threads:
        t.join()
      
      # Do memory action if found in memory. Also, do a replay to learn the transition of the start state to the end state
      if not best_minions.empty():
        depth, action, minion_state_history, minion_action_history, final_action = best_minions.get()
        # print(f'Found best action {action}. Depth {depth}. Current state {env.cur_state}.')

        if depth > 0:
          # update fast neural network with each state on future state trajectory as start state, and the final goal state as the goal state, with each action on future action trajectory
          start_states.extend([x+goal_state for x in minion_state_history])
          start_action.extend([int(x) for x in minion_action_history])

    # perform action
    obs, rewards, done, _= env.step(action)
    next_state = (obs[0], obs[1])

    # STEP 4: store transition in memory, delete irrelevant states if environment has changed
    for mem in [memory, episode_memory]:
      if len(mem[cur_state][action]) > 0 and mem[cur_state][action][-1] != next_state:
        mem[cur_state][action] = [next_state]
      else:
        mem[cur_state][action].append(next_state)

    # append current state and action to history
    statehistory.append(cur_state)
    actionhistory.append(action)

    # terminate if completed
    if done:
      if rewards == 1 and steps+1 < grid_size*grid_size:
        print(f'Run {i+1} completed in {steps+1} steps. Minimum steps: {env.min_steps}, Start state: {env.start_state}, Goal state: {env.goal_state}')

      else:
        print(f'Run {i+1} not completed. Minimum steps: {env.min_steps}, Start state: {env.start_state}, Goal state: {env.goal_state}')
      best_steps.append(env.min_steps)
      actual_steps.append(steps+1)

      break

    # STEP 5: Backward Replay: update fast neural network with next state as goal state, and states before as start state, with current action
    if use_fast:
      start_states.extend([x+next_state for x in statehistory])
      start_action.extend([int(x) for x in actionhistory])
      model.fit(start_states, start_action, epochs=1, verbose = False)

In [ ]:
plt.plot(best_steps, 'b-', label = 'Minimum Steps per Episode')
plt.plot(actual_steps, 'g-', label = 'Actual Steps per Episode')
plt.xlabel('Episode number')
plt.ylabel('Number of Steps')
plt.xticks([i*10 for i in range(11)])
# plt.legend(loc = 'upper right')
plt.show()

actual_steps = np.array(actual_steps)
print(f'Number of steps above minimum for first 50 epochs is {sum(actual_steps[:50])-sum(best_steps[:50])}')
print(f'Number of steps above minimum for last 50 epochs is {sum(actual_steps[50:])-sum(best_steps[50:])}')
print(f'Number of steps above minimum overall is {sum(actual_steps)-sum(best_steps)}')
print(f'Number of episodes solved for first 50 epochs is {sum(actual_steps[:50]<grid_size*grid_size)}')
print(f'Number of episodes solved for last 50 epochs is {sum(actual_steps[50:]<grid_size*grid_size)}')
print(f'Number of episodes solved overall is {sum(actual_steps<grid_size*grid_size)}')

## Model 4: Without fast goal-directed mechanism

In [ ]:
# hyperparameters
use_fast = False
use_slow = True

grid_size = 10
exploration_const = 1
parallel_threads = 100
lookahead_depth = 20

# create model
model = GetModel()

# reset memory
memory = defaultdict(lambda: defaultdict(lambda: []))

# store only first action
best_steps = []
actual_steps = []

for i in range(100):
  env = GridEnv(grid_size, start_state = None, goal_state = None, forbidden_squares = GetGrid(i, grid_size), seed = i)
  statehistory = []
  actionhistory = []
  episode_memory = defaultdict(lambda: defaultdict(lambda: []))

  for steps in range(grid_size*grid_size):
    cur_state = env.cur_state
    goal_state = env.goal_state

    # initialize training array
    start_states = []
    start_action = []
    # STEP 1: Fast neural network to get general cue for direction
    if use_fast:
      action_prob = model.predict(np.array(cur_state+goal_state).reshape(-1,4), verbose = 0)
      action_probs = action_prob[0]
    else:
      # set to 0s if not using the fast neural network
      action_probs = np.array([0 for _ in range(num_actions)], dtype = 'float')

    # STEP 2: Do explore-exploit to choose action

    # calculate the exploration parameter, take action_probs and add in count values from instances in memory
    action_probs -= [exploration_const*np.sqrt(len(episode_memory[env.cur_state][i])) for i in range(num_actions)]
    
    # choose the best action via explore-exploit tradeoff
    action = np.argmax(action_probs)

    # STEP 3: Forward Replay: find out if there is a path in memory to the goal (akin to Model-Based Planning)
    if use_slow:
      best_minions = PriorityQueue()
      best_action = 0

      threads = []
      sem = threading.Semaphore()
      for _ in range(parallel_threads):
        t = Thread(target=minion())
        threads.append(t)
        t.start()

      for t in threads:
        t.join()
      
      # Do memory action if found in memory. Also, do a replay to learn the transition of the start state to the end state
      if not best_minions.empty():
        depth, action, minion_state_history, minion_action_history, final_action = best_minions.get()
        # print(f'Found best action {action}. Depth {depth}. Current state {env.cur_state}.')

        if depth > 0:
          # update fast neural network with each state on future state trajectory as start state, and the final goal state as the goal state, with each action on future action trajectory
          start_states.extend([x+goal_state for x in minion_state_history])
          start_action.extend([int(x) for x in minion_action_history])

    # perform action
    obs, rewards, done, _= env.step(action)
    next_state = (obs[0], obs[1])

    # STEP 4: store transition in memory, delete irrelevant states if environment has changed
    for mem in [memory, episode_memory]:
      if len(mem[cur_state][action]) > 0 and mem[cur_state][action][-1] != next_state:
        mem[cur_state][action] = [next_state]
      else:
        mem[cur_state][action].append(next_state)

    # append current state and action to history
    statehistory.append(cur_state)
    actionhistory.append(action)

    # terminate if completed
    if done:
      if rewards == 1 and steps+1 < grid_size*grid_size:
        print(f'Run {i+1} completed in {steps+1} steps. Minimum steps: {env.min_steps}, Start state: {env.start_state}, Goal state: {env.goal_state}')

      else:
        print(f'Run {i+1} not completed. Minimum steps: {env.min_steps}, Start state: {env.start_state}, Goal state: {env.goal_state}')
      best_steps.append(env.min_steps)
      actual_steps.append(steps+1)

      break

    # STEP 5: Backward Replay: update fast neural network with next state as goal state, and states before as start state, with current action
    if use_fast:
      start_states.extend([x+next_state for x in statehistory])
      start_action.extend([int(x) for x in actionhistory])
      model.fit(start_states, start_action, epochs=1, verbose = False)

In [ ]:
plt.plot(best_steps, 'b-', label = 'Minimum Steps per Episode')
plt.plot(actual_steps, 'g-', label = 'Actual Steps per Episode')
plt.xlabel('Episode number')
plt.ylabel('Number of Steps')
plt.xticks([i*10 for i in range(11)])
# plt.legend(loc = 'upper right')
plt.show()

actual_steps = np.array(actual_steps)
print(f'Number of steps above minimum for first 50 epochs is {sum(actual_steps[:50])-sum(best_steps[:50])}')
print(f'Number of steps above minimum for last 50 epochs is {sum(actual_steps[50:])-sum(best_steps[50:])}')
print(f'Number of steps above minimum overall is {sum(actual_steps)-sum(best_steps)}')
print(f'Number of episodes solved for first 50 epochs is {sum(actual_steps[:50]<grid_size*grid_size)}')
print(f'Number of episodes solved for last 50 epochs is {sum(actual_steps[50:]<grid_size*grid_size)}')
print(f'Number of episodes solved overall is {sum(actual_steps<grid_size*grid_size)}')

## Model 5: Without slow memory and fast goal-directed mechanism
Just explore-exploit

In [ ]:
# hyperparameters
use_fast = False
use_slow = False

grid_size = 10
exploration_const = 1
parallel_threads = 100
lookahead_depth = 20

# create model
model = GetModel()

# reset memory
memory = defaultdict(lambda: defaultdict(lambda: []))

# store only first action
best_steps = []
actual_steps = []

for i in range(100):
  env = GridEnv(grid_size, start_state = None, goal_state = None, forbidden_squares = GetGrid(i, grid_size), seed = i)
  statehistory = []
  actionhistory = []
  episode_memory = defaultdict(lambda: defaultdict(lambda: []))

  for steps in range(grid_size*grid_size):
    cur_state = env.cur_state
    goal_state = env.goal_state

    # initialize training array
    start_states = []
    start_action = []
    # STEP 1: Fast neural network to get general cue for direction
    if use_fast:
      action_prob = model.predict(np.array(cur_state+goal_state).reshape(-1,4), verbose = 0)
      action_probs = action_prob[0]
    else:
      # set to 0s if not using the fast neural network
      action_probs = np.array([0 for _ in range(num_actions)], dtype = 'float')

    # STEP 2: Do explore-exploit to choose action

    # calculate the exploration parameter, take action_probs and add in count values from instances in memory
    action_probs -= [exploration_const*np.sqrt(len(episode_memory[env.cur_state][i])) for i in range(num_actions)]
    
    # choose the best action via explore-exploit tradeoff
    action = np.argmax(action_probs)

    # STEP 3: Forward Replay: find out if there is a path in memory to the goal (akin to Model-Based Planning)
    if use_slow:
      best_minions = PriorityQueue()
      best_action = 0

      threads = []
      sem = threading.Semaphore()
      for _ in range(parallel_threads):
        t = Thread(target=minion())
        threads.append(t)
        t.start()

      for t in threads:
        t.join()
      
      # Do memory action if found in memory. Also, do a replay to learn the transition of the start state to the end state
      if not best_minions.empty():
        depth, action, minion_state_history, minion_action_history, final_action = best_minions.get()
        # print(f'Found best action {action}. Depth {depth}. Current state {env.cur_state}.')

        if depth > 0:
          # update fast neural network with each state on future state trajectory as start state, and the final goal state as the goal state, with each action on future action trajectory
          start_states.extend([x+goal_state for x in minion_state_history])
          start_action.extend([int(x) for x in minion_action_history])

    # perform action
    obs, rewards, done, _= env.step(action)
    next_state = (obs[0], obs[1])

    # STEP 4: store transition in memory, delete irrelevant states if environment has changed
    for mem in [memory, episode_memory]:
      if len(mem[cur_state][action]) > 0 and mem[cur_state][action][-1] != next_state:
        mem[cur_state][action] = [next_state]
      else:
        mem[cur_state][action].append(next_state)

    # append current state and action to history
    statehistory.append(cur_state)
    actionhistory.append(action)

    # terminate if completed
    if done:
      if rewards == 1:
        print(f'Run {i+1} completed in {steps+1} steps. Minimum steps: {env.min_steps}, Start state: {env.start_state}, Goal state: {env.goal_state}')
        best_steps.append(env.min_steps)
        actual_steps.append(steps+1)

      else:
        print(f'Run {i+1} not completed. Minimum steps: {env.min_steps}, Start state: {env.start_state}, Goal state: {env.goal_state}')
        best_steps.append(env.min_steps)
        actual_steps.append(steps+1)

      break

    # STEP 5: Backward Replay: update fast neural network with next state as goal state, and states before as start state, with current action
    if use_fast:
      start_states.extend([x+next_state for x in statehistory])
      start_action.extend([int(x) for x in actionhistory])
      model.fit(start_states, start_action, epochs=1, verbose = False)

In [ ]:
plt.plot(best_steps, 'b-', label = 'Minimum Steps per Episode')
plt.plot(actual_steps, 'g-', label = 'Actual Steps per Episode')
plt.xlabel('Episode number')
plt.ylabel('Number of Steps')
plt.xticks([i*10 for i in range(11)])
# plt.legend(loc = 'upper right')
plt.show()

actual_steps = np.array(actual_steps)
print(f'Number of steps above minimum for first 50 epochs is {sum(actual_steps[:50])-sum(best_steps[:50])}')
print(f'Number of steps above minimum for last 50 epochs is {sum(actual_steps[50:])-sum(best_steps[50:])}')
print(f'Number of steps above minimum overall is {sum(actual_steps)-sum(best_steps)}')
print(f'Number of episodes solved for first 50 epochs is {sum(actual_steps[:50]<grid_size*grid_size)}')
print(f'Number of episodes solved for last 50 epochs is {sum(actual_steps[50:]<grid_size*grid_size)}')
print(f'Number of episodes solved overall is {sum(actual_steps<grid_size*grid_size)}')

# Ablation Studies
- Vary lookahead depth
- Vary number of parallel branch

In [ ]:
for lookahead_depth in [5, 10, 50]:

    # hyperparameters
    use_fast = True
    use_slow = True

    grid_size = 10
    exploration_const = 1
    parallel_threads = 100
    # lookahead_depth = 20

    # create model
    model = GetModel()

    # reset memory
    memory = defaultdict(lambda: defaultdict(lambda: []))

    # store only first action
    best_steps = []
    actual_steps = []

    for i in range(100):
      env = GridEnv(grid_size, start_state = None, goal_state = None, forbidden_squares = GetGrid(i, grid_size), seed = i)
      statehistory = []
      actionhistory = []
      episode_memory = defaultdict(lambda: defaultdict(lambda: []))

      for steps in range(grid_size*grid_size):
        cur_state = env.cur_state
        goal_state = env.goal_state

        # initialize training array
        start_states = []
        start_action = []
        # STEP 1: Fast neural network to get general cue for direction
        if use_fast:
          action_prob = model.predict(np.array(cur_state+goal_state).reshape(-1,4), verbose = 0)
          action_probs = action_prob[0]
        else:
          # set to 0s if not using the fast neural network
          action_probs = np.array([0 for _ in range(num_actions)], dtype = 'float')

        # STEP 2: Do explore-exploit to choose action

        # calculate the exploration parameter, take action_probs and add in count values from instances in memory
        action_probs -= [exploration_const*np.sqrt(len(episode_memory[env.cur_state][i])) for i in range(num_actions)]
        
        # choose the best action via explore-exploit tradeoff
        action = np.argmax(action_probs)

        # STEP 3: Forward Replay: find out if there is a path in memory to the goal (akin to Model-Based Planning)
        if use_slow:
          best_minions = PriorityQueue()
          best_action = 0

          threads = []
          sem = threading.Semaphore()
          for _ in range(parallel_threads):
            t = Thread(target=minion())
            threads.append(t)
            t.start()

          for t in threads:
            t.join()
          
          # Do memory action if found in memory. Also, do a replay to learn the transition of the start state to the end state
          if not best_minions.empty():
            depth, action, minion_state_history, minion_action_history, final_action = best_minions.get()
            # print(f'Found best action {action}. Depth {depth}. Current state {env.cur_state}.')

            if depth > 0:
              # update fast neural network with each state on future state trajectory as start state, and the final goal state as the goal state, with each action on future action trajectory
              start_states.extend([x+goal_state for x in minion_state_history])
              start_action.extend([int(x) for x in minion_action_history])

        # perform action
        obs, rewards, done, _= env.step(action)
        next_state = (obs[0], obs[1])

        # STEP 4: store transition in memory, delete irrelevant states if environment has changed
        for mem in [memory, episode_memory]:
          if len(mem[cur_state][action]) > 0 and mem[cur_state][action][-1] != next_state:
            mem[cur_state][action] = [next_state]
          else:
            mem[cur_state][action].append(next_state)

        # append current state and action to history
        statehistory.append(cur_state)
        actionhistory.append(action)

        # terminate if completed
        if done:
          if rewards == 1:
            print(f'Run {i+1} completed in {steps+1} steps. Minimum steps: {env.min_steps}, Start state: {env.start_state}, Goal state: {env.goal_state}')
            best_steps.append(env.min_steps)
            actual_steps.append(steps+1)

          else:
            print(f'Run {i+1} not completed. Minimum steps: {env.min_steps}, Start state: {env.start_state}, Goal state: {env.goal_state}')
            best_steps.append(env.min_steps)
            actual_steps.append(steps+1)

          break

        # STEP 5: Backward Replay: update fast neural network with next state as goal state, and states before as start state, with current action
        if use_fast:
          start_states.extend([x+next_state for x in statehistory])
          start_action.extend([int(x) for x in actionhistory])
          model.fit(start_states, start_action, epochs=1, verbose = False)

    plt.plot(best_steps, 'b-', label = 'Minimum Steps per Episode')
    plt.plot(actual_steps, 'g-', label = 'Actual Steps per Episode')
    plt.xlabel('Episode number')
    plt.ylabel('Number of Steps')
    plt.xticks([i*10 for i in range(11)])
    # plt.legend(loc = 'upper right')
    plt.show()

    actual_steps = np.array(actual_steps)
    print(f'Number of steps above minimum for first 50 epochs is {sum(actual_steps[:50])-sum(best_steps[:50])}')
    print(f'Number of steps above minimum for last 50 epochs is {sum(actual_steps[50:])-sum(best_steps[50:])}')
    print(f'Number of steps above minimum overall is {sum(actual_steps)-sum(best_steps)}')
    print(f'Number of episodes solved for first 50 epochs is {sum(actual_steps[:50]<grid_size*grid_size)}')
    print(f'Number of episodes solved for last 50 epochs is {sum(actual_steps[50:]<grid_size*grid_size)}')
    print(f'Number of episodes solved overall is {sum(actual_steps<grid_size*grid_size)}')

In [ ]:
for parallel_threads in [10, 50, 200]:

    # hyperparameters
    use_fast = True
    use_slow = True

    grid_size = 10
    exploration_const = 1
    # parallel_threads = 100
    lookahead_depth = 20

    # create model
    model = GetModel()

    # reset memory
    memory = defaultdict(lambda: defaultdict(lambda: []))

    # store only first action
    best_steps = []
    actual_steps = []

    for i in range(100):
      env = GridEnv(grid_size, start_state = None, goal_state = None, forbidden_squares = GetGrid(i, grid_size), seed = i)
      statehistory = []
      actionhistory = []
      episode_memory = defaultdict(lambda: defaultdict(lambda: []))

      for steps in range(grid_size*grid_size):
        cur_state = env.cur_state
        goal_state = env.goal_state

        # initialize training array
        start_states = []
        start_action = []
        # STEP 1: Fast neural network to get general cue for direction
        if use_fast:
          action_prob = model.predict(np.array(cur_state+goal_state).reshape(-1,4), verbose = 0)
          action_probs = action_prob[0]
        else:
          # set to 0s if not using the fast neural network
          action_probs = np.array([0 for _ in range(num_actions)], dtype = 'float')

        # STEP 2: Do explore-exploit to choose action

        # calculate the exploration parameter, take action_probs and add in count values from instances in memory
        action_probs -= [exploration_const*np.sqrt(len(episode_memory[env.cur_state][i])) for i in range(num_actions)]
        
        # choose the best action via explore-exploit tradeoff
        action = np.argmax(action_probs)

        # STEP 3: Forward Replay: find out if there is a path in memory to the goal (akin to Model-Based Planning)
        if use_slow:
          best_minions = PriorityQueue()
          best_action = 0

          threads = []
          sem = threading.Semaphore()
          for _ in range(parallel_threads):
            t = Thread(target=minion())
            threads.append(t)
            t.start()

          for t in threads:
            t.join()
          
          # Do memory action if found in memory. Also, do a replay to learn the transition of the start state to the end state
          if not best_minions.empty():
            depth, action, minion_state_history, minion_action_history, final_action = best_minions.get()
            # print(f'Found best action {action}. Depth {depth}. Current state {env.cur_state}.')

            if depth > 0:
              # update fast neural network with each state on future state trajectory as start state, and the final goal state as the goal state, with each action on future action trajectory
              start_states.extend([x+goal_state for x in minion_state_history])
              start_action.extend([int(x) for x in minion_action_history])

        # perform action
        obs, rewards, done, _= env.step(action)
        next_state = (obs[0], obs[1])

        # STEP 4: store transition in memory, delete irrelevant states if environment has changed
        for mem in [memory, episode_memory]:
          if len(mem[cur_state][action]) > 0 and mem[cur_state][action][-1] != next_state:
            mem[cur_state][action] = [next_state]
          else:
            mem[cur_state][action].append(next_state)

        # append current state and action to history
        statehistory.append(cur_state)
        actionhistory.append(action)

        # terminate if completed
        if done:
          if rewards == 1:
            print(f'Run {i+1} completed in {steps+1} steps. Minimum steps: {env.min_steps}, Start state: {env.start_state}, Goal state: {env.goal_state}')
            best_steps.append(env.min_steps)
            actual_steps.append(steps+1)

          else:
            print(f'Run {i+1} not completed. Minimum steps: {env.min_steps}, Start state: {env.start_state}, Goal state: {env.goal_state}')
            best_steps.append(env.min_steps)
            actual_steps.append(steps+1)

          break

        # STEP 5: Backward Replay: update fast neural network with next state as goal state, and states before as start state, with current action
        if use_fast:
          start_states.extend([x+next_state for x in statehistory])
          start_action.extend([int(x) for x in actionhistory])
          model.fit(start_states, start_action, epochs=1, verbose = False)

    plt.plot(best_steps, 'b-', label = 'Minimum Steps per Episode')
    plt.plot(actual_steps, 'g-', label = 'Actual Steps per Episode')
    plt.xlabel('Episode number')
    plt.ylabel('Number of Steps')
    plt.xticks([i*10 for i in range(11)])
    # plt.legend(loc = 'upper right')
    plt.show()

    actual_steps = np.array(actual_steps)
    print(f'Number of steps above minimum for first 50 epochs is {sum(actual_steps[:50])-sum(best_steps[:50])}')
    print(f'Number of steps above minimum for last 50 epochs is {sum(actual_steps[50:])-sum(best_steps[50:])}')
    print(f'Number of steps above minimum overall is {sum(actual_steps)-sum(best_steps)}')
    print(f'Number of episodes solved for first 50 epochs is {sum(actual_steps[:50]<grid_size*grid_size)}')
    print(f'Number of episodes solved for last 50 epochs is {sum(actual_steps[50:]<grid_size*grid_size)}')
    print(f'Number of episodes solved overall is {sum(actual_steps<grid_size*grid_size)}')